## Preprocessing

In [19]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [20]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

df = application_df.drop(['EIN', 'NAME'], axis=1)

In [21]:
# Determine the number of unique values in each column.
unique_values = df.nunique()

# Print the number of unique values in each column
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [22]:
# Look at APPLICATION_TYPE value counts for binning
app_type = df['APPLICATION_TYPE'].value_counts()

# Print the value counts
print(app_type)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [23]:
# Choose a cutoff value and create a list of application types to be replaced
cutoff_value = 500  

application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

# Replace in the dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check the value counts after binning
application_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [24]:
# Look at CLASSIFICATION value counts for binning

classification_counts = application_df['CLASSIFICATION'].value_counts()
#PRINT

print(classification_counts)


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [25]:
# You may find it helpful to look at CLASSIFICATION value counts >1

classification_counts = application_df['CLASSIFICATION'].value_counts()

classification_counts_greater_than_1 = classification_counts[classification_counts > 1]

print(classification_counts_greater_than_1)


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [26]:

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

cutoff_value = 1800

classification_counts = df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts <= cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [27]:
# Convert categorical data to numeric with `pd.get_dummies`
# Identify the categorical columns
categorical_data = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']

# Convert categorical columns to dummy variables
dummy_variables = pd.get_dummies(df[categorical_data])

# Concatenate the dummy variables with the original DataFrame
application_df_numeric = pd.concat([df, dummy_variables], axis=1)

# Drop the original categorical columns if desired
application_df_numeric = application_df_numeric.drop(categorical_data, axis=1)


In [28]:
# Split our preprocessed data into our features and target arrays
X = application_df_numeric.drop('IS_SUCCESSFUL', axis=1)
y = application_df_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

In [36]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train_scaled.shape[1]
hidden_nodes_layer1 =  8000
hidden_nodes_layer2 = 300
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="LeakyReLU"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 8000)              408000    
                                                                 
 dense_9 (Dense)             (None, 300)               2400300   
                                                                 
 dense_10 (Dense)            (None, 10)                3010      
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 2,811,321
Trainable params: 2,811,321
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [38]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 25s 30ms/step - loss: 0.5798 - accuracy: 0.7182
Epoch 2/200
804/804 [==============================] - 24s 30ms/step - loss: 0.5616 - accuracy: 0.7263
Epoch 3/200
804/804 [==============================] - 24s 30ms/step - loss: 0.5580 - accuracy: 0.7274
Epoch 4/200
804/804 [==============================] - 24s 30ms/step - loss: 0.5567 - accuracy: 0.7280
Epoch 5/200
804/804 [==============================] - 24s 30ms/step - loss: 0.5567 - accuracy: 0.7269
Epoch 6/200
804/804 [==============================] - 24s 30ms/step - loss: 0.5545 - accuracy: 0.7267
Epoch 7/200
804/804 [==============================] - 24s 30ms/step - loss: 0.5545 - accuracy: 0.7310
Epoch 8/200
804/804 [==============================] - 24s 30ms/step - loss: 0.5518 - accuracy: 0.7304
Epoch 9/200
804/804 [==============================] - 24s 30ms/step - loss: 0.5508 - accuracy: 0.7308
Epoch 10/200
804/804 [==============================] - 24s 30ms/step - l

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5787 - accuracy: 0.7311 - 535ms/epoch - 2ms/step
Loss: 0.5787014365196228, Accuracy: 0.7310787439346313


In [35]:
# Export our model to HDF5 file
AlphabetSoupCharity_Optimization = "AlphabetSoupCharity_Optimization.h5"
nn.save(AlphabetSoupCharity_Optimization)
